# AI 악성코드 탐지 프로젝트 최종 보고서

# -정보보호와시스템보안 프로젝트-


팀   이   름 : M(make) and M(modify)

조        원 : 변승훈, 진세광

제   출   일 : 2020-11-27

담 당  교 수 : 윤 명 근

# -목차-

1.  Project 개요

2.  Feature Vector Extraction

  -PEMINER Feature Vector

  -EMBER Feature Vector

  -PESTUDIO Feature Vector


3. 각 모델로 Accuracy(Score) 산출


4. Hyper Parameter Search


5. 앙상블 적용


6. 최종 결과


7. 참고 문헌


8. 감상 및 소감


# 1. Project 개요

**-개요-**

step1.

peminer, ember, pestudio 로 추출 된 pe정보를 가공하여 고정 크기의 벡터 피처로 만든다.


step2. 

가공 한 학습데이터에 여러 머신러닝 알고리즘을 적용하여 각각의 모델을 생성한다.


step3. 

생성한 모델들로 검증데이터를 predict하여 정확성을 확인하고, 가장 좋은 성적을 갖은 모델을 선택한다.

step4.

선택한 모델의 알고리즘의 파라미터를 조정하여 학습성능을 높힌다.

step5.

최종 모델들을 앙상블하여 테스트데이터를 predict한다.

-

**-목표-**

데이터의 피처를 추출하여 머신러닝으로 학습하여 만든 모델로 테스트 데이터의 predict를 얻어 낸다.


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: PESTUDIO/학습데이터/bd9cf98f928470d06332223ddd84b0ac9af055a54a07775b75a497b4cc947919.json  
  inflating: PESTUDIO/학습데이터/bda2ab110f80f6fa79b3d650b6ae4565d8e840d7ba5b69e38c2874f63cc423a8.json  
  inflating: PESTUDIO/학습데이터/bda37d4ed18f33df81f03823e62707da458120a021d0878d6c450b5c0f157f3f.json  
  inflating: PESTUDIO/학습데이터/bdaa46116f28aeae7c99d9d18a06d9ed3181ed82a6e719fb0956e9672c2012cf.json  
  inflating: PESTUDIO/학습데이터/bdab51b550107ca68011548a4a84aae5115dcf9d9210e91cd9bfde15406a4a64.json  
  inflating: PESTUDIO/학습데이터/bdaf361591e90d89a9adbb3d070f27635029b705fcbbd99f7cfed738262e0d2b.json  
  inflating: PESTUDIO/학습데이터/bdb0b50f0281fedb6477b5202e460c0e6613a650f5352f5ddc4aa6ce5cb570b5.json  
  inflating: PESTUDIO/학습데이터/bdb251111c8b6ee237092544abbcc81daa8c6e237459bd497fe8f840f6d24d12.json  
  inflating: PESTUDIO/학습데이터/bdb46fc3b901bbecaf104504ea723df0512f77d037343c91e6db0f731c00d048.json  
  inflating: PESTUDIO/학습데이터/bdb60ce6b039af2a7842b56c24dcd5a88d32

In [ ]:
pip install scikit-optimize

     |████████████████████████████████| 102kB 3.8MB/s 


In [ ]:
import os
import glob
import json
import pprint
import csv

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import lightgbm as lgb

from lightgbm import LGBMClassifier
from lightgbm import plot_importance

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.feature_extraction import FeatureHasher

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold

from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.plots import plot_convergence

# 2.Feature Vector Extraction

PEMINER

PEMINER의 모든 특징들을 모두 벡터에 추가해 만들었습니다.

-
    
EMBER

실수 또는 바이너리 형태의 데이터는 바로 피처 벡터에 추가 했습니다.

Feature Hashing을 이용하여 가변 집합, 스트링을 고정 크기 벡터로 변환하여 추가했습니다.

-

PESTUDIO


**overview**

overview의 실수형태인 size값과 entropy값을 가져오는데 size값은 너무크다고 판단해서 가져올때 10000으로 나누어서 가져오도록했습니다.

**indicator**

indicator를 Feature Hashing을 이용해 고정 크기 벡터로 변환했습니다. 이때 고정 크기 벡터의 크기는 indicator의 길이에서 가장 많이 분포한 개수의 값을 가져왔습니다.
**virustotal**

virustotal의 탐지 수를 가져옵니다.

**dos**

dos-header : entropy,size
    
dos-stub : entropy, size

file-header : size-of-optional-header

optional-header : size-of-header

위의 6개의 특징을 가져옵니다.

**resources**

instance의 값들을 Feature Hashing을 이용해서 고정 크기 벡터로 변환하고 벡터 크기는 instance의 개수 중 가장 많이 분포한 개수를 가져왔습니다.




In [ ]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):    
        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        return self.vector
    

class EmberParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()

    def get_byteentropy_info(self):
        byteentropy = np.array(self.report["byteentropy"])
        total = byteentropy.sum()
        vector = byteentropy / total
        return vector.tolist()
    
    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    def get_datadirectories_info(self):
        datadirectories = self.report["datadirectories"]
        vector = []
        if datadirectories == []:
          vector = [0] * 30
        else:
          for i in range(len(datadirectories)):
            vector += datadirectories[i]["size"], datadirectories[i]["virtual_address"]
        return vector

    def get_sections_info(self):
        raw_obj = self.report["section"]
        sections = raw_obj["sections"]
        general = [
            len(sections), 
            sum(1 for s in sections if s['size'] == 0),
            sum(1 for s in sections if s['name'] == ""),
            sum(1 for s in sections if 'MEM_READ' in s['props'] and 'MEM_EXECUTE' in s['props']),
            sum(1 for s in sections if 'MEM_WRITE' in s['props'])
        ]
        section_sizes = [(s['name'], s['size']) for s in sections]
        section_sizes_hashed = FeatureHasher(50, input_type="pair").transform([section_sizes]).toarray()[0]
        section_entropy = [(s['name'], s['entropy']) for s in sections]
        section_entropy_hashed = FeatureHasher(50, input_type="pair").transform([section_entropy]).toarray()[0]
        section_vsize = [(s['name'], s['vsize']) for s in sections]
        section_vsize_hashed = FeatureHasher(50, input_type="pair").transform([section_vsize]).toarray()[0]
        entry_name_hashed = FeatureHasher(50, input_type="string").transform([raw_obj['entry']]).toarray()[0]
        characteristics = [p for s in sections for p in s['props'] if s['name'] == raw_obj['entry']]
        characteristics_hashed = FeatureHasher(50, input_type="string").transform([characteristics]).toarray()[0]

        return np.hstack([
            general, section_sizes_hashed, section_entropy_hashed, section_vsize_hashed, entry_name_hashed,
            characteristics_hashed
        ]).astype(np.float32).tolist()

    def get_header_info(self):
        raw_obj = self.report["header"]
        return np.hstack([
            raw_obj['coff']['timestamp'],
            FeatureHasher(10, input_type="string").transform([[raw_obj['coff']['machine']]]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([raw_obj['coff']['characteristics']]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([[raw_obj['optional']['subsystem']]]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([raw_obj['optional']['dll_characteristics']]).toarray()[0],
            FeatureHasher(10, input_type="string").transform([[raw_obj['optional']['magic']]]).toarray()[0],
            raw_obj['optional']['major_image_version'],
            raw_obj['optional']['minor_image_version'],
            raw_obj['optional']['major_linker_version'],
            raw_obj['optional']['minor_linker_version'],
            raw_obj['optional']['major_operating_system_version'],
            raw_obj['optional']['minor_operating_system_version'],
            raw_obj['optional']['major_subsystem_version'],
            raw_obj['optional']['minor_subsystem_version'],
            raw_obj['optional']['sizeof_code'],
            raw_obj['optional']['sizeof_headers'],
            raw_obj['optional']['sizeof_heap_commit'],
        ]).astype(np.float32).tolist()

    def get_imports_info(self):
        raw_obj = self.report["imports"]
        libraries = list(set([l.lower() for l in raw_obj.keys()]))
        libraries_hashed = FeatureHasher(256, input_type="string").transform([libraries]).toarray()[0]

        imports = [lib.lower() + ':' + e for lib, elist in raw_obj.items() for e in elist]
        imports_hashed = FeatureHasher(1024, input_type="string").transform([imports]).toarray()[0]

        return np.hstack([libraries_hashed, imports_hashed]).astype(np.float32).tolist()

    def get_exports_info(self):
        raw_obj = self.report["exports"]
        exports_hashed = FeatureHasher(128, input_type="string").transform([raw_obj]).toarray()[0]
        return exports_hashed.astype(np.float32).tolist()

    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        vector += self.get_histogram_info()
        vector += self.get_datadirectories_info()
        vector += self.get_string_info()
        vector += self.get_sections_info()
        vector += self.get_byteentropy_info()
        vector += self.get_header_info()
        vector += self.get_imports_info()
        vector += self.get_exports_info()

        return vector
    
class PestudioParser:
    '''
        사용할 특징을 선택하여 벡터화 할 것을 권장
    '''
    
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    def overview(self) :
          size = float(self.report['image']['overview']['size']) / 10000
          entropy = float(self.report['image']['overview']['entropy'])
          vector = [size,entropy]
          return vector
    def indicator(self) :
        vector_hashed = []
        if len(self.report['image']) < 20 :
            vector_hashed = [-1 for x in range(50)]
        else :
          if len(self.report['image']['indicators']) !=1 :
            vector = [instance for instance in self.report['image']['indicators']['indicator']]
            vector_hashed = FeatureHasher(50).transform(vector).toarray()[0].astype(np.float32).tolist()
          else :
            vector = [self.report['image']['indicators']['indicator']]
            vector_hashed = FeatureHasher(50).transform(vector).toarray()[0].astype(np.float32).tolist()
        return vector_hashed
    def virustotal(self) :
        if type(self.report['image']['virustotal']) != dict :
           vector = [-1]
        else :
           vector = [float(self.report['image']['virustotal']['@detection'])]
    def dos(self) :
        if len(self.report['image']) < 20 :
          vector = [-1,-1,-1,-1,-1,-1]
        else :
          vector = [
                   float(self.report['image']['dos-header']['entropy']),
                   float(self.report['image']['dos-header']['size']),
                   float(self.report['image']['dos-stub']['entropy']),
                   float(self.report['image']['dos-stub']['size']),
                   float(self.report['image']['file-header']['size-of-optional-header']),
                   float(self.report['image']['optional-header']['size-of-headers'])
                   ]
        return vector
    def resource_extraction(self) :
      if len(self.report['image']['resources'])==2 :
        if len(self.report['image']['resources']['instance']) !=10 :
           vector = [instance for instance in self.report['image']['resources']['instance']]
           vector_hashed = FeatureHasher(51).transform(vector).toarray()[0].astype(np.float32).tolist()
        else :
           vector = [self.report['image']['resources']['instance']]
           vector_hashed = FeatureHasher(51).transform(vector).toarray()[0].astype(np.float32).tolist()
      else :
           vector_hashed = [-1 for x in range(51)]
      return vector_hashed
    def process_report(self) :
        vector = []
        vector += self.overview()
        vector += self.indicator()
        vector += self.virustotal()
        vector += self.dos()
        vector += self.resource_extraction()
        return vector

# 3. 각 모델로 Accuracy(Score) 산출

**파라미터 튜닝 및 앙상블 전 Accuracy(Score)**

LightGBM : 96.24%

RandomForest :94.86%

DecisionTree : 91.79%

SVM :82.99%

Logistic Regression :82.43%

KNeighbors :90.42%

AdaBoost :91.86%

MLP : 85.48%


In [ ]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding="cp949") as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_estimators=100, n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"],
          n_estimators=1800, num_leaves= 90, min_data_in_leaf = 22
          )
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))
    return predict

def predict_result(X_pre, y_pre, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_pre)
    table = [y_pre, predict.tolist()]

    return table


In [ ]:
X, y = [], []
label_table = read_label_csv("학습데이터_정답.csv")

for fname in os.listdir("/content/PEMINER/학습데이터"):
    feature_vector = []
    lname = fname.split('.')[0]
    label = label_table[lname]
    for data in ["/content/PEMINER/학습데이터", "/content/EMBER/학습데이터","/content/PESTUDIO/학습데이터"]:
        path = f"{data}/{fname}"
        if data == "/content/PEMINER/학습데이터":
            feature_vector += PeminerParser(path).process_report()
        elif data == "/content/EMBER/학습데이터":
            feature_vector += EmberParser(path).process_report()
        else:
          try :
            feature_vector += PestudioParser(path).process_report()
          except :
            feature_vector += [-1 for x in range(110)]
    X.append(feature_vector)
    y.append(label)

np.asarray(X).shape, np.asarray(y).shape

((20000, 2679), (20000,))

In [ ]:
X_test, y_test = [], []
label_table = read_label_csv("검증데이터_정답.csv")

for fname in os.listdir("/content/PEMINER/검증데이터"):
    feature_vector = []
    lname = fname.split('.')[0]
    label = label_table[lname]
    for data in ["/content/PEMINER/검증데이터", "/content/EMBER/검증데이터","/content/PESTUDIO/검증데이터"]:
        path = f"{data}/{fname}"
        if data == "/content/PEMINER/검증데이터":
            feature_vector += PeminerParser(path).process_report()
        elif data == "/content/EMBER/검증데이터":
            feature_vector += EmberParser(path).process_report()
        else:
          try :
            feature_vector += PestudioParser(path).process_report()
          except :
            feature_vector += [-1 for x in range(110)]
    X_test.append(feature_vector)
    y_test.append(label)

np.asarray(X_test).shape, np.asarray(y_test).shape

((10000, 2679), (10000,))

In [ ]:
models = []
for model in ["lgb", "rf", "dt", "svm", "lr", "knn", "adaboost", "mlp"]:
    clf = train(X, y, model)
    models.append(clf)

In [ ]:
for model in models:
    evaluate(X_test, y_test, model)

정확도 0.9624
정확도 0.9486
정확도 0.9179
정확도 0.8299
정확도 0.8243
정확도 0.9042
정확도 0.9186
정확도 0.8548


# 4. Hyper Parameter Search

모델들 중에서 score 우선순위에 따라서 LightGBM과 RandomForest가 높았습니다. 이 두 모델들의 파라미터를 튜닝해 더 높은 Accuracy를 산출하고 앙상블 시키려고 하였습니다.

가장 최적의 성능을 보이는 모델의 파라미터 값을 찾기위해 직관적으로 값들을 넣어보거나 GridSearch, 베이지안 옵티미제이션을 이용했습니다.

-

-

**-RandomForest-**
    
Parameters

n_estimators 	 = 결정 트리의 개수를 지정

max_depth    	 = 트리의 최대 깊이

min_samples_leaf   =  리프노드가 되기 위한 최소한의 샘플 데이터수

min_samples_split  = 노드를 분할하기위한 최소한의 샘플 데이터의 수

-

n_estimators          : 100		/		default : 10

max_depth             : [6,8,10,12]		/	default : None

min_samples_leaf  : [8,12,18]		/		default : 1

min_samples_split : [8, 16 20]	/		default : 2

-

정확성 : 94.02%

결과값이 처음 정확도에 비해서 오히려 떨어졌다는것을 확인하고 default값이 가장 정확도가 높게 나왔습니다.

-

-

**-Light GBM-**


n_estimators          :  1800				

max_depth             :  default (-1)

num_leaves           :  90

min_data_in_leaf   :  22

정확성 : 0.9675

여러 파라미터 값들과 범위로 그리드서치, 베이지안 옵티미제이션를 진행했을 때 max_depth 값을 기본값인 -1이 아닌 다른 값을 주게 되었을경우 정확성이 많이 떨어지는 것을 확인했습니다. 

피처가 많기 때문에 max_depth를 제한 없이 분기하도록 하고 과적합이 나지 않는 수준에서 학습하기 위해  n_estimators값을 1800정도로 조절 했습니다. 

num_leaves은 max_depth에 맞춰서 줘야하는데 여러번 시도한 결과 90이 가장 성능이 좋았고 min_data_in_leaf을 기본 값인 20보다 조금 높게줘서 과적합을 줄였습니다.


In [ ]:
# RandomForest GridSearch
parameters = { 'n_estimators' : [100,200],
           'max_depth' : [None, 6, 8, 10, 12],
           'min_samples_leaf' : [1,8, 12, 18],
           'min_samples_split' : [2, 8, 16, 20]
            }
rf_clf = RandomForestClassifier(random_state=SEED)
grid_cv = GridSearchCV(rf_clf, param_grid=parameters, cv=3, n_jobs=4)

grid_cv.fit(X, y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
search_space = [Real(0.4, 0.7, name='learning_rate'),
                Integer(50, 70, name='max_depth'),
                Integer(1000, 1800, name='num_leaves'),
                Integer(1, 5, name='scale_pos_weight'),
                Integer(20, 50, name='min_child_samples')
                ]

def BayesianOptimization(values):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': values[0],
        'max_depth': values[1],
        'nthread': -1,
        'num_leaves': values[2],
        'scale_pos_weight': values[3],
        'min_child_samples': values[4]
    }

    print('\nTesting next set of paramaters...', params)

    train_data = lgb.Dataset(X, label=y)
    valid_data = lgb.Dataset(X_test, label=y_test)

    lgbm = lgb.train(params, train_data, 10, valid_sets=valid_data, early_stopping_rounds=50,
                 verbose_eval=0)

    neg_auc = round(-roc_auc_score(y_test, lgbm.predict(X_test)), 6)

    print('AUC: ', -neg_auc, ' of boosting iteration ', lgbm.current_iteration())
    return neg_auc

In [ ]:
res_gp = gp_minimize(BayesianOptimization, search_space, random_state=26, n_jobs=-1, verbose=True, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.6139333699331987, 'max_depth': 56, 'nthread': -1, 'num_leaves': 1447, 'scale_pos_weight': 1, 'min_child_samples': 23}
AUC:  0.989699  of boosting iteration  10
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 40.2731
Function value obtained: -0.9897
Current minimum: -0.9897
Iteration No: 2 started. Evaluating function at random point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.4678336991433171, 'max_depth': 65, 'nthread': -1, 'num_leaves': 1652, 'scale_pos_weight': 3, 'min_child_samples': 32}
AUC:  0.988641  of boosting iteration  10
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 33.9789
Function value obtained: -0.9886
Current minimum: -0.9897
Iteration No: 3 started. Evaluating function at 

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


AUC:  0.989295  of boosting iteration  10
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 29.6769
Function value obtained: -0.9893
Current minimum: -0.9897
Iteration No: 23 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.4189609654713921, 'max_depth': 50, 'nthread': -1, 'num_leaves': 1764, 'scale_pos_weight': 5, 'min_child_samples': 50}
AUC:  0.988496  of boosting iteration  10
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 28.6795
Function value obtained: -0.9885
Current minimum: -0.9897
Iteration No: 24 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.6079482190882682, 'max_depth': 50, 'nthread': -1, 'num_leaves': 1001, 'scale_pos_weight': 1, 'min_child_samples': 50}
AUC:  0.988607  of b

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


AUC:  0.98915  of boosting iteration  10
Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 42.5908
Function value obtained: -0.9891
Current minimum: -0.9897
Iteration No: 36 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.7, 'max_depth': 70, 'nthread': -1, 'num_leaves': 1800, 'scale_pos_weight': 1, 'min_child_samples': 20}
AUC:  0.98915  of boosting iteration  10
Iteration No: 36 ended. Search finished for the next optimal point.
Time taken: 43.7556
Function value obtained: -0.9891
Current minimum: -0.9897
Iteration No: 37 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.7, 'max_depth': 50, 'nthread': -1, 'num_leaves': 1000, 'scale_pos_weight': 1, 'min_child_samples': 50}
AUC:  0.988984  of boosting iteration  10
Iteration 

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


AUC:  0.98915  of boosting iteration  10
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 42.7320
Function value obtained: -0.9891
Current minimum: -0.9906
Iteration No: 43 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.7, 'max_depth': 50, 'nthread': -1, 'num_leaves': 1000, 'scale_pos_weight': 5, 'min_child_samples': 20}
AUC:  0.988609  of boosting iteration  10
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 39.7322
Function value obtained: -0.9886
Current minimum: -0.9906
Iteration No: 44 started. Searching for the next optimal point.

Testing next set of paramaters... {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.42164862033656086, 'max_depth': 50, 'nthread': -1, 'num_leaves': 1049, 'scale_pos_weight': 5, 'min_child_samples': 22}
AUC:  0.988335  of boosting iterati

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


KeyboardInterrupt: ignored

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                    test_size=0.2, #기본:0.25
                                                    stratify=y, #target의 class비율에 맞춰서 분리
                                                    random_state=1)

np.asarray(X_train).shape, np.asarray(y_train).shape, np.asarray(X_val).shape, np.asarray(y_val).shape

params = {'max_depth': [15, 20, 40],
          'min_child_samples': [20, 40, 60],
          'subsample': [0.6, 0.8, 1]}
lgbm = LGBMClassifier()
grid = GridSearchCV(lgbm, param_grid=params)
grid.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc',
         eval_set=[(X_train, y_train), (X_val, y_val)])

print("최적 파라미터: ", grid.best_params_)


Streaming output truncated to the last 5000 lines.
[48]	valid_0's binary_logloss: 0.0697062	valid_0's auc: 0.998016	valid_1's binary_logloss: 0.111088	valid_1's auc: 0.991857
[49]	valid_0's binary_logloss: 0.0682746	valid_0's auc: 0.9981	valid_1's binary_logloss: 0.110147	valid_1's auc: 0.991957
[50]	valid_0's binary_logloss: 0.0668482	valid_0's auc: 0.998178	valid_1's binary_logloss: 0.109378	valid_1's auc: 0.991999
[51]	valid_0's binary_logloss: 0.0654533	valid_0's auc: 0.998257	valid_1's binary_logloss: 0.108397	valid_1's auc: 0.9921
[52]	valid_0's binary_logloss: 0.0642321	valid_0's auc: 0.998328	valid_1's binary_logloss: 0.107804	valid_1's auc: 0.992101
[53]	valid_0's binary_logloss: 0.0630535	valid_0's auc: 0.998383	valid_1's binary_logloss: 0.107372	valid_1's auc: 0.992102
[54]	valid_0's binary_logloss: 0.0618642	valid_0's auc: 0.998443	valid_1's binary_logloss: 0.106882	valid_1's auc: 0.992141
[55]	valid_0's binary_logloss: 0.0606804	valid_0's auc: 0.998511	valid_1's binary_log

NameError: ignored

#  5. 앙상블 적용

softvoting방법을 이용해 파라미터를 최적화 시킨 lgbm과 rf를 앙상블 한 결과 오히려 lgbm 하나만을 이용하여 predict 한 것보다 정확성이 떨어지는 것을 확인했습니다.

lgbm과 rf의 정확성차이가 심하기 때문으로 판단하고 최종적으로는 앙상블을 하지않고 lgbm만을 이용하여 테스트데이터를 predict하기로 결정했습니다.



In [ ]:
models = []
for model in ["lgb", "rf"]:
    clf = (model, load_model(model=model, random_state=SEED))
    models.append(clf)

soft_vote  = VotingClassifier(models, voting='soft')

soft_vote.fit(X, y)

evaluate(X_test, y_test, soft_vote)

정확도 0.9659


# 6. 최종 결과

가장 높은 Accuracy 산출 모델 : LightGBM
                    
정확도 = 96.75%

2번째로 높은 Accuracy 산출 모델 : RandomForest

정확도 = 94.86%

Predict.csv:
https://github.com/wwoss3650/MandM


In [ ]:
X_pre, y_pre = [], []

for fname in os.listdir("/content/PEMINER/테스트데이터"):
    feature_vector = []
    lname = fname.split('.')[0]
    label = lname
    for data in ["/content/PEMINER/테스트데이터", "/content/EMBER/테스트데이터","/content/PESTUDIO/테스트데이터"]:
        path = f"{data}/{fname}"
        if data == "/content/PEMINER/테스트데이터":
            feature_vector += PeminerParser(path).process_report()
        elif data == "/content/EMBER/테스트데이터":
            feature_vector += EmberParser(path).process_report()
        else:
          try :
            feature_vector += PestudioParser(path).process_report()
          except :
            feature_vector += [-1 for x in range(110)]
    X_pre.append(feature_vector)
    y_pre.append(label)

np.asarray(X_pre).shape, np.asarray(y_pre).shape

((10000, 2679), (10000,))

In [ ]:
for model in models:
    result = predict_result(X_pre, y_pre, model)

print(result)

[['...'], [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,

# 7. 참고 문헌
sklearn.esemble.RandomForestClassifier - 
         https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

sklearnFeatureHasher - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html#sklearn.feature_extraction.FeatureHasher

RandomForest 참고-데이터분석, 머신러닝 정리 노트 - https://injo.tistory.com/30

sklearn.esemble.LightGBM - https://lightgbm.readthedocs.io/en/latest/Parameters.html

RandomForest 유튜브 영상 - https://www.youtube.com/watch?v=J4Wdy0Wc_xQ



# 8. 감상 및 소감
     

**변승훈 :** 

처음 AI 악성코드 탐지프로젝트를 귀에 접했을때 무엇을 어떻게해야될지 감도 못 잡고 두려워했습니다. 하지만 같은 조원인 진세광 학우님과 함께 AI와 악성코드 탐지에 대해 토론하고 관련 사이트에서 공부하면서 하나하나씩 이걸 풀어내면서 수수께끼를 풀어내는것과같이 재미있게 프로젝트를 진행할 수 있었습니다. 또한 부족한 점이 있다면 역시 AI와 정보보호에 관한 전문지식들이 많이 부족 하다는걸 깨닫게 되고 지금의 공부량보다 더 많이 노력해야 된다는걸 새삼 맛보게 됬습니다. 학부에서 이런 프로젝트를 접할 수 있는 기회가 많지 않습니다. 이런 기회를 주신 교수님께 감사를 표하고 slack에서 질문하면 성심성의껏 대답해주신 조교님께도 감사를 표합니다. 또한 저와 프로젝트를 함께 진행한 진세광 학우님이 가장 감사한 사람입니다. 긴 프로젝트를 하느라 수고하셨습니다.


-


**진세광 :**

이번 프로젝트에서는 pe정보를 peminer, ember, pestudio로 추출 한 것을 받아서 진행했지만 나중에는 직접 추출 해보는 것도 좋을 것 같습니다.

추출 된 데이터들을 가공 할 때, 단순히 벡터로 변환 시키는 것이 아니라 피처들이 의미를 갖을 수 있도록 추출하고 정규화 하는 등 피처 엔지니어링의 중요성을 알 수 있었습니다. 피처 중요도를 확인해 봤을 때, 아무리 중요도가 낮더라도 다른 피처와의 연계성이 있는 경우가 많아서 빼버리게 되면 모델을 학습했을 때 정확성이 떨어지는 경우가 많았습니다.

LGBM은 Data set이 많을 경우에 높은 성능을 내는 것을 확인 할 수 있었고 각각의 파라미터 들을 조정하여 더 좋은 성능을 이끌어 낼 수 있음을 확인 하였습니다. 물론 의미 있는 성능의 차이가 났지만, 파라미터 조정보다 피처 엔지니어링이 정확성에 더 크게 기여한다고 느꼈습니다.

RF와 LGBM의 정확성이 차이가 많이 나서 앙상블의 효율이 좋지 않았는데, 피처를 좀 더 뽑아내거나 최적 파라미터를 더 연구하는 등의 방법으로 RF의 성능을 좀 더 끌어올려 앙상블까지 수행했다면 더 좋은 결과를 낼 수 있지 않았을까 하는 아쉬움이 남습니다.

pe파일의 헤더 정보와 ML, 피처엔지니어링 등 많은 것을 공부해 볼 수 있는 프로젝트 였고 다음에는 딥러닝도 함께 이용해보고 싶습니다. 변승훈 학우와 서로 도와가며 함께 공부 한 덕분에 많은 도움이 되었습니다.
